# Семантично ядро

В този примерен код ще използвате [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework, за да създадете основен агент.

Целта на този пример е да ви покаже стъпките, които по-късно ще използваме в допълнителните примерни кодове при реализирането на различните агентни модели.


## Импортирайте необходимите Python пакети


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Създаване на клиента

В този пример ще използваме [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) за достъп до LLM.

`ai_model_id` е дефиниран като `gpt-4o-mini`. Опитайте да промените модела на друг, наличен в GitHub Models marketplace, за да видите различни резултати.

За да използваме `Azure Inference SDK`, който се използва за `base_url` за GitHub Models, ще използваме конектора `OpenAIChatCompletion` в Semantic Kernel. Съществуват и други [налични конектори](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), които позволяват използването на Semantic Kernel за други доставчици на модели.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Създаване на Агента

По-долу създаваме Агента, наречен `TravelAgent`.

В този пример използваме много прости инструкции. Можете да промените тези инструкции, за да видите как агентът реагира по различен начин.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Стартиране на агента

Сега можем да стартираме агента, като дефинираме нишка от тип `ChatHistoryAgentThread`. Всички необходими системни съобщения се предоставят на аргумента `messages` на invoke_stream.

След като тези са дефинирани, създаваме `user_inputs`, което представлява това, което потребителят изпраща на агента. В този случай сме задали това съобщение на `Планирай ми слънчева ваканция`.

Чувствайте се свободни да промените това съобщение, за да видите как агентът реагира различно.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
